<a href="https://colab.research.google.com/github/Rustydinoco/ML_Learning/blob/master/Simple_Neural_Network_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:

# Create a Model Class that inherit nn.Module

class Model(nn.Module):
# Input Layer = 4
# First Hidden Layer h1 = 6
# Second Hidden Layer h2 = 8
# output Layer = 3
  def __init__(self, in_features=4, h1=6, h2=8, out_features=3):
    super().__init__()
    self.fc1= nn.Linear(in_features, h1)
    self.fc2= nn.Linear(h1, h2)
    self.fc3= nn.Linear(h2, out_features)

# Activation the Linear with ReLU function
  def forward (self , x):
    x= F.relu(self.fc1(x))
    x= F.relu(self.fc2(x))
    x= self.out(x)
    return x



In [12]:
# Pick a Manual Seed for Randomizatibn
torch.manual_seed(41)
# Create an instance of Model
model= Model()

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
df = pd.read_csv(url)

In [16]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [17]:
df['species']=df['species'].replace('setosa', 0.0)
df['species']=df['species'].replace('versicolor', 1.0)
df['species']=df['species'].replace('virginica', 2.0)

<ipython-input-17-051aa53064c0>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['species']=df['species'].replace('virginica', 2.0)


In [18]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [19]:
X = df.drop('species', axis= 1)
y = df['species']

In [20]:
X = X.values
y = y.values